In [1]:
import os
import pandas as pd
import numpy as np
import sys
from pathlib import Path

In [2]:
project_dir = Path(os.getcwd()).parent # goes from notebooks to mri-classifier
src_dir = os.path.join(project_dir, 'src')
# add src directory for access to helper functions
sys.path.append(src_dir)

In [3]:
from helper import *

In [4]:
# paths
print(project_dir)
# have a folder of dcm images in this path
#dcm_folder = '/Volumes/Samsung_T5/FinalProject/TCGA-GBM'
dcm_folder = 'D:\\MRI\\TCGA-GBM'
print(dcm_folder)

C:\Users\ikara\Documents\Propulsion\mri-classifier
D:\MRI\TCGA-GBM


In [5]:
dcim_images = find_dcm_images(dcm_folder)
print(len(dcim_images))
# example
print(list(dcim_images[0:1]))

481158
['D:\\MRI\\TCGA-GBM\\TCGA-02-0003\\06-08-1997-MRI BRAIN WWO CONTRAMR-81239\\6.000000-epiRT - LHAND-02082\\1-01.dcm']


In [6]:
substr = 'COR'
cor_list = [img for img in dcim_images if substr in img] 
#p = [print(cor) for cor in cor_list]
print(len(cor_list))

11867


In [7]:
substr = 'SAG'
sag_list = [img for img in dcim_images if substr in img] 
print(len(sag_list))

14603


In [8]:
substr = 'AX '
ax_list = [img for img in dcim_images if substr in img] 
print(len(ax_list))

27782


In [11]:
import json
with open(os.path.join(src_dir, 'mri-images.json')) as jsonfile:
    data = json.load(jsonfile)

In [10]:
s = 'TCGA120616'
s2 = f'{s[0:4]}-{s[4:6]}-{s[6:]}'
s2

'TCGA-12-0616'

In [12]:
create_patient_id = lambda s: f'{s[0:4]}-{s[4:6]}-{s[6:]}'
patient_ids = [create_patient_id(data[key]['patient-id']) for key in data.keys()]
patient_ids[:5]

['TCGA-12-0616',
 'TCGA-12-0616',
 'TCGA-76-4934',
 'TCGA-76-4934',
 'TCGA-76-6661']

In [15]:
def get_new_patients(patient_ids):
    """
    Checks all patient folder names and saves those in a list which are not already in the json
     - patient_ids: list of patient ids from the json
    """
    all_patients = [f.name for f in os.scandir(dcm_folder) if f.is_dir()]
    new_patients = [patient for patient in all_patients if patient not in patient_ids]
    return new_patients

get_new_patients(patient_ids)[:10]

['TCGA-06-0127',
 'TCGA-06-0129',
 'TCGA-06-0130',
 'TCGA-06-0132',
 'TCGA-06-0133',
 'TCGA-06-0137',
 'TCGA-06-0138',
 'TCGA-06-0139',
 'TCGA-06-0141',
 'TCGA-06-0142']

In [74]:
test_patient_ids = lambda s: [c not in s for c in patient_ids]
#test_patient_ids('TCGA-02-0003')
#newcor_list = [cor for cor in cor_list if test_patient_ids(cor)]
#newcor_list
s = 'TCGA-02-0003'
def test_filename(filename, patient_ids):
    for c in patient_ids:
        if c in filename:
            return True
    return False

filename = '/Volumes/Samsung_T5/FinalProject/TCGA-GBM/TCGA-06-0185/11-07-2003-26945/13.000000-GD COR T1-25076/1-01.dcm'
test_filename(filename, patient_ids)

False

In [75]:
newcor_list = [c for c in cor_list if not test_filename(c, patient_ids)]
newcor_list

['/Volumes/Samsung_T5/FinalProject/TCGA-GBM/TCGA-02-0034/07-27-1997-MRI BRAIN WWO CONTRAMR-39956/14.000000-COR T1 POST-51672/1-01.dcm',
 '/Volumes/Samsung_T5/FinalProject/TCGA-GBM/TCGA-02-0034/07-27-1997-MRI BRAIN WWO CONTRAMR-39956/14.000000-COR T1 POST-51672/1-02.dcm',
 '/Volumes/Samsung_T5/FinalProject/TCGA-GBM/TCGA-02-0034/07-27-1997-MRI BRAIN WWO CONTRAMR-39956/14.000000-COR T1 POST-51672/1-03.dcm',
 '/Volumes/Samsung_T5/FinalProject/TCGA-GBM/TCGA-02-0034/07-27-1997-MRI BRAIN WWO CONTRAMR-39956/14.000000-COR T1 POST-51672/1-04.dcm',
 '/Volumes/Samsung_T5/FinalProject/TCGA-GBM/TCGA-02-0034/07-27-1997-MRI BRAIN WWO CONTRAMR-39956/14.000000-COR T1 POST-51672/1-05.dcm',
 '/Volumes/Samsung_T5/FinalProject/TCGA-GBM/TCGA-02-0034/07-27-1997-MRI BRAIN WWO CONTRAMR-39956/14.000000-COR T1 POST-51672/1-06.dcm',
 '/Volumes/Samsung_T5/FinalProject/TCGA-GBM/TCGA-02-0034/07-27-1997-MRI BRAIN WWO CONTRAMR-39956/14.000000-COR T1 POST-51672/1-07.dcm',
 '/Volumes/Samsung_T5/FinalProject/TCGA-GBM/TCGA

In [76]:
print(len(newcor_list))
print(len(cor_list))

8931
11867


In [26]:
tdf = load_json_as_df(data_dir, 'mri-images')

loaded mri-images.json with shape (229, 6)



In [27]:
tdf.head()

,patient-id,perspective,sequence,perspective-sequence,image-number,image-name
0,TCGA120616,AX,ADC,AX-ADC,1,AX_ADC_Glioma_GBM_TCGA120616_1.jpg
1,TCGA120616,AX,ADC,AX-ADC,2,AX_ADC_Glioma_GBM_TCGA120616_2.jpg
2,TCGA764934,AX,ADC,AX-ADC,1,AX_ADC_Glioma_GBM_TCGA764934_1.jpg
3,TCGA764934,AX,ADC,AX-ADC,2,AX_ADC_Glioma_GBM_TCGA764934_2.jpg
4,TCGA766662,AX,ADC,AX-ADC,1,AX_ADC_Glioma_GBM_TCGA766662_1.jpg


In [77]:
col = 'perspective-sequence'
classes_df = tdf.groupby(by=[col])
available_classes = set(classes_df.groups.keys())
print(f'Number of available classes: {len(available_classes)}')
p = [print(f'Class: {key}\t Size: {len(classes_df.get_group(key))}') for key in available_classes]

Number of available classes: 15
Class: AX-PWI	 Size: 7
Class: AX-ADC	 Size: 10
Class: AX-T1	 Size: 25
Class: COR-T2	 Size: 5
Class: COR-T1	 Size: 2
Class: SAG-T1	 Size: 26
Class: AX-Misc	 Size: 12
Class: AX-T2	 Size: 39
Class: COR-Misc	 Size: 3
Class: SAG-T1KM	 Size: 16
Class: AX-PD	 Size: 5
Class: AX-T1KM	 Size: 22
Class: COR-T1KM	 Size: 33
Class: AX-DWI	 Size: 21
Class: SAG-Misc	 Size: 3


In [78]:
col = 'perspective'
classes_df = tdf.groupby(by=[col])
available_classes = set(classes_df.groups.keys())
print(f'Number of available classes: {len(available_classes)}')
p = [print(f'Class: {key}\t Size: {len(classes_df.get_group(key))}') for key in available_classes]

Number of available classes: 3
Class: AX	 Size: 141
Class: SAG	 Size: 45
Class: COR	 Size: 43


In [81]:
substr = 'Perfusion'
sag_list = [img for img in dcim_images if substr.lower() in img.lower()] 
print(len(sag_list))

106256
